In [2]:
import sys

sys.path.append(r"g:\Other computers\Corporativa\WebScraping\busqueda_bdf\tools")

# Chunk 2: Importa librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


# Importar el módulo de similitud
from product_similarity import (
    evaluar_producto_scraping,
    seleccionar_mejor_sustituto,
    calcular_similitud_descripcion,
    comparar_marcas,
    analizar_resultados_scraping,
    imprimir_resumen_analisis
)


In [3]:
import pandas as pd
import numpy as np

In [4]:
lista_articulos = pd.read_csv("../data/lista_articulos.csv")

In [5]:
lista_articulos

,descripcion,marca
0,Tape eléctrico Súper 33 19mm x 20.1m Scotch,3M
1,Tomacorriente doble polariz tipo americano lín...,Eagle
2,Plástico negro x kilo ancho: 4 mts espesor: 5gg,Termoencogibles de CR
3,Plafón light blanco,Eagle
4,Tape eléctrico Temflex 165 19mm x 18 m Temflex,3M
...,...,...
1037,Hidrolavadora 1595 PSI 1400W con ruedas,Daewoo
1038,Modulo 4 medidores fijo MP 125A barras 400A MP...,Schneider Electric
1039,Rotomartillo 1700W 19 Joules 12Kg VV SDS max H...,Bosch
1040,Silla plegable 54 x 43 x 80.5cm blanca,Basic Living


In [ ]:
# --------- Chunk 7: Columnas objetivo actualizadas ---------
columnas = [
    "keyword",        # lo que buscas
    "description",    # nombre producto
    "cod",           # código del producto
    "brand",         # marca
    "price",         # precio actual
    "old_price",     # precio anterior (si hay promoción)
    "type_promotion", # Oportunidad, Liquidación, ""
    "start_date",    # fecha inicio promo
    "end_date",      # fecha fin promo
    "reference",     # referencia técnica
    "barra",         # código de barras
    "is_substitute",  # "target" o "substitute"
    "match_score",   # puntuación de similitud (0-100)
    "url"            # url del producto
]

LIMIT = None # <-- Cambia para testing, pon None para full luego


In [ ]:
# --------- Chunk 8: Función de extracción detalle mejorada ---------
def extraer_detalle_epa_mejorado(driver, url):
    """Extrae todos los detalles del producto de una URL"""
    import time
    import re
    from selenium.webdriver.common.by import By

    driver.get(url)
    time.sleep(2)

    # Nombre del producto
    try:
        nombre = driver.find_element(By.CSS_SELECTOR, "h1.page-title span").text.strip()
    except:
        nombre = ""

    # Precio actual
    try:
        price_elem = driver.find_element(By.CSS_SELECTOR, "span.price")
        price_text = price_elem.text.strip()
        precio = price_text.replace('¢', '').replace(' ', '').replace('.', '')
    except:
        precio = ""

    # Código de producto
    try:
        codigo = driver.find_element(By.CSS_SELECTOR, "div.product.attribute.sku div.value").text.strip()
    except:
        codigo = ""

    # Marca
    marca = ""
    try:
        try:
            tab = driver.find_element(By.ID, "tab-label-additional")
            if "active" not in tab.get_attribute("class"):
                tab.click()
                time.sleep(1)
        except Exception:
            pass
        rows = driver.find_elements(By.CSS_SELECTOR, "#additional table tbody tr")
        for row in rows:
            th = row.find_element(By.CSS_SELECTOR, "th").text.strip().lower()
            td = row.find_element(By.CSS_SELECTOR, "td").text.strip()
            if th == "marca":
                marca = td
                break
    except:
        marca = ""

    # Detección de promociones
    tipo_promocion = ""
    precio_anterior = ""
    fecha_inicio = ""
    fecha_fin = ""

    # Primero intentar detectar el tipo de promoción
    tipo_promocion_temp = ""
    try:
        promo_label = driver.find_element(By.CSS_SELECTOR, "span.price-label")
        tipo_promocion_temp = promo_label.text.strip()
    except:
        tipo_promocion_temp = ""

    # Extraer precio anterior
    try:
        old_price_elem = driver.find_element(By.CSS_SELECTOR, "span[id^='old-price-'] span.price")
        old_price_text = old_price_elem.text.strip()
        precio_anterior = old_price_text.replace('¢', '').replace(' ', '').replace('.', '')
    except:
        precio_anterior = ""

    # Extraer fechas de promoción
    try:
        promo_desc_elem = driver.find_element(By.CSS_SELECTOR, "div.product.attribute.descripcion_promo div.value")
        fecha_texto = promo_desc_elem.text.strip()

        if tipo_promocion_temp.lower() == "liquidación":
            match = re.search(r'Desde\s+(\d{2}/\d{2}/\d{4})', fecha_texto)
            if match:
                fecha_fin = match.group(1)
                fecha_inicio = ""
        elif tipo_promocion_temp.lower() == "oportunidad":
            match = re.search(r'Desde\s+(\d{2}/\d{2}/\d{4})\s+hasta\s+(\d{2}/\d{2}/\d{4})', fecha_texto)
            if match:
                fecha_inicio = match.group(1)
                fecha_fin = match.group(2)
    except:
        pass

    # VALIDACIÓN: Solo asignar tipo_promocion si realmente hay fechas O precio anterior
    if fecha_fin or precio_anterior:
        tipo_promocion = tipo_promocion_temp
    else:
        tipo_promocion = ""  # No hay promoción real

    referencia = "S/R"
    barra = "S/B"

    return {
        "nombre": nombre,
        "precio": precio,
        "codigo": codigo,
        "marca": marca,
        "precio_anterior": precio_anterior,
        "tipo_promocion": tipo_promocion,
        "fecha_inicio": fecha_inicio,
        "fecha_fin": fecha_fin,
        "referencia": referencia,
        "barra": barra
    }

In [ ]:
# --------- Chunk 4: Setup driver headless ---------
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


driver = web_driver()


In [ ]:
# --------- Chunk 10: Loop principal mejorado con lógica target/sustituto, DataFrames incrementales y reanudación por última búsqueda ---------
import os
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Archivos para guardado incremental
validos_path = 'df_validos_incremental.csv'
descartados_path = 'df_descartados_incremental.csv'
busquedas_path = 'busquedas_incremental.csv'

# Cargar progreso de búsquedas si existe
if os.path.exists(busquedas_path):
    df_busquedas = pd.read_csv(busquedas_path)
    if not df_busquedas.empty:
        ultima_busqueda = df_busquedas.iloc[-1]['descripcion']
        # Buscar el índice de la última búsqueda en lista_articulos
        idx_ultima = lista_articulos[lista_articulos['descripcion'] == ultima_busqueda].index
        if len(idx_ultima) > 0:
            start_idx = idx_ultima[0] + 1
        else:
            start_idx = 0
    else:
        start_idx = 0
else:
    df_busquedas = pd.DataFrame(columns=['descripcion'])
    start_idx = 0

# Cargar dataframes incrementales si existen
if os.path.exists(validos_path):
    df_validos = pd.read_csv(validos_path)
    validos = df_validos.to_dict('records')
else:
    validos = []
if os.path.exists(descartados_path):
    df_descartados = pd.read_csv(descartados_path)
    descartados = df_descartados.to_dict('records')
else:
    descartados = []

productos_guardados = []
articulos_sin_match = []

for idx, row in lista_articulos.iloc[start_idx:].head(LIMIT).iterrows():
    actual_idx = idx + start_idx
    descripcion_buscada = row['descripcion']
    marca_buscada = row.get('marca', '').strip()

    print(f"\n{'='*60}")
    print(f"Buscando: '{descripcion_buscada}' (índice {actual_idx})")
    print(f"Marca objetivo: '{marca_buscada}'")

    # Realizar búsqueda en EPA
    EPA_URL = "https://cr.epaenlinea.com/"
    driver.get(EPA_URL)
    time.sleep(2)

    barra = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "aa-Input"))
    )
    barra.clear()
    barra.send_keys(descripcion_buscada)
    barra.send_keys(Keys.ENTER)
    time.sleep(1.2)

    # Obtener solo los primeros 2 resultados
    links = driver.find_elements(By.CSS_SELECTOR, "a.result")
    urls = [link.get_attribute("href") for link in links[:2]]
    print(f"Productos encontrados: {len(urls)}")

    if not urls:
        print(f"  ⚠️ No se encontraron resultados para '{descripcion_buscada}'")
        articulos_sin_match.append(descripcion_buscada)
        continue

    # FASE 1: Buscar el producto TARGET
    producto_target = None
    productos_evaluados = []

    for i, url in enumerate(urls):
        try:
            print(f"\n  Evaluando resultado #{i+1}...")
            datos_producto = extraer_detalle_epa_mejorado(driver, url)

            # Calcular similitud usando función personalizada
            score_similitud = calcular_similitud_descripcion(
                descripcion_buscada,
                datos_producto['nombre']
            )

            # Verificar marca
            marca_coincide = comparar_marcas(marca_buscada, datos_producto['marca'])

            print(f"    Nombre: {datos_producto['nombre'][:50]}...")
            print(f"    Marca encontrada: {datos_producto['marca']}")
            print(f"    Score similitud: {score_similitud:.1f}%")
            print(f"    Marca coincide: {'✓' if marca_coincide else '✗'}")

            # Guardar para evaluación
            productos_evaluados.append({
                'url': url,
                'datos': datos_producto,
                'score': score_similitud,
                'marca_coincide': marca_coincide,
                'indice': i
            })

            # CRITERIOS PARA PRODUCTO TARGET:
            # 1. Primera prioridad: Marca coincide Y similitud > 60%
            # 2. Segunda prioridad: Solo primer resultado si similitud > 80% (aunque marca no coincida)
            if marca_coincide and score_similitud > 60:
                producto_target = productos_evaluados[-1]
                print(f"    ✓✓ PRODUCTO TARGET ENCONTRADO (marca y similitud)")
                break
            elif i == 0 and score_similitud > 80:
                producto_target = productos_evaluados[-1]
                print(f"    ✓ PRODUCTO TARGET ENCONTRADO (alta similitud, posición 1)")
                break
            elif i == 0 and not marca_buscada and score_similitud > 60:
                producto_target = productos_evaluados[-1]
                print(f"    ✓ PRODUCTO TARGET ENCONTRADO (sin marca específica)")
                break

        except Exception as e:
            print(f"    ✗ Error evaluando producto: {e}")
            continue

    # Si no encontramos target en los primeros resultados, skip este artículo
    if not producto_target:
        print(f"\n  ⚠️ No se encontró producto target válido para '{descripcion_buscada}'")
        print(f"     Posible razón: Marca '{marca_buscada}' no encontrada o baja similitud")
        articulos_sin_match.append(descripcion_buscada)
        for prod in productos_evaluados:
            prod_row = {
                "keyword": descripcion_buscada,
                "description": prod['datos']['nombre'],
                "cod": prod['datos']['codigo'],
                "brand": prod['datos']['marca'],
                "price": prod['datos']['precio'],
                "old_price": prod['datos']['precio_anterior'],
                "type_promotion": prod['datos']['tipo_promocion'],
                "start_date": prod['datos']['fecha_inicio'],
                "end_date": prod['datos']['fecha_fin'],
                "reference": prod['datos']['referencia'],
                "barra": prod['datos']['barra'],
                "is_substitute": "descartado",
                "match_score": f"{prod['score']:.1f}",
                "url": prod['url']
            }
            descartados.append(prod_row)
        # Guardar progreso cada 100 productos
        if (actual_idx+1) % 100 == 0:
            pd.DataFrame(validos).to_csv(validos_path, index=False)
            pd.DataFrame(descartados).to_csv(descartados_path, index=False)
            df_busquedas = pd.concat([df_busquedas, pd.DataFrame([{ 'descripcion': descripcion_buscada }])], ignore_index=True)
            df_busquedas.to_csv(busquedas_path, index=False)
        continue

    # Guardar producto TARGET
    producto_final_target = {
        "keyword": descripcion_buscada,
        "description": producto_target['datos']['nombre'],
        "cod": producto_target['datos']['codigo'],
        "brand": producto_target['datos']['marca'],
        "price": producto_target['datos']['precio'],
        "old_price": producto_target['datos']['precio_anterior'],
        "type_promotion": producto_target['datos']['tipo_promocion'],
        "start_date": producto_target['datos']['fecha_inicio'],
        "end_date": producto_target['datos']['fecha_fin'],
        "reference": producto_target['datos']['referencia'],
        "barra": producto_target['datos']['barra'],
        "is_substitute": "target",
        "match_score": f"{producto_target['score']:.1f}",
        "url": producto_target['url']
    }
    validos.append(producto_final_target)
    productos_guardados.append(producto_final_target)
    print(f"\n  ✓ TARGET guardado: {producto_target['datos']['nombre'][:50]}...")

    # FASE 2: Buscar SUSTITUTO
    producto_sustituto = None
    mejor_sustituto_score = 0

    for prod in productos_evaluados:
        if prod == producto_target:
            continue
        es_marca_diferente = not comparar_marcas(
            producto_target['datos']['marca'],
            prod['datos']['marca']
        )
        if es_marca_diferente and prod['score'] > 50:
            if prod['score'] > mejor_sustituto_score:
                mejor_sustituto_score = prod['score']
                producto_sustituto = prod

    if producto_sustituto:
        producto_final_sustituto = {
            "keyword": descripcion_buscada,
            "description": producto_sustituto['datos']['nombre'],
            "cod": producto_sustituto['datos']['codigo'],
            "brand": producto_sustituto['datos']['marca'],
            "price": producto_sustituto['datos']['precio'],
            "old_price": producto_sustituto['datos']['precio_anterior'],
            "type_promotion": producto_sustituto['datos']['tipo_promocion'],
            "start_date": producto_sustituto['datos']['fecha_inicio'],
            "end_date": producto_sustituto['datos']['fecha_fin'],
            "reference": producto_sustituto['datos']['referencia'],
            "barra": producto_sustituto['datos']['barra'],
            "is_substitute": "substitute",
            "match_score": f"{producto_sustituto['score']:.1f}",
            "url": producto_sustituto['url']
        }
        validos.append(producto_final_sustituto)
        productos_guardados.append(producto_final_sustituto)
        print(f"  ✓ SUSTITUTO guardado: {producto_sustituto['datos']['nombre'][:50]}...")
    else:
        print(f"  ⚠️ No se encontró sustituto válido")

    # Guardar los productos no seleccionados como descartados
    for prod in productos_evaluados:
        if prod == producto_target or prod == producto_sustituto:
            continue
        prod_row = {
            "keyword": descripcion_buscada,
            "description": prod['datos']['nombre'],
            "cod": prod['datos']['codigo'],
            "brand": prod['datos']['marca'],
            "price": prod['datos']['precio'],
            "old_price": prod['datos']['precio_anterior'],
            "type_promotion": prod['datos']['tipo_promocion'],
            "start_date": prod['datos']['fecha_inicio'],
            "end_date": prod['datos']['fecha_fin'],
            "reference": prod['datos']['referencia'],
            "barra": prod['datos']['barra'],
            "is_substitute": "descartado",
            "match_score": f"{prod['score']:.1f}",
            "url": prod['url']
        }
        descartados.append(prod_row)

    # Guardar progreso cada 100 productos
    if (actual_idx+1) % 100 == 0:
        pd.DataFrame(validos).to_csv(validos_path, index=False)
        pd.DataFrame(descartados).to_csv(descartados_path, index=False)
        df_busquedas = pd.concat([df_busquedas, pd.DataFrame([{ 'descripcion': descripcion_buscada }])], ignore_index=True)
        df_busquedas.to_csv(busquedas_path, index=False)

# Guardar al final también
pd.DataFrame(validos).to_csv(validos_path, index=False)
pd.DataFrame(descartados).to_csv(descartados_path, index=False)
df_busquedas = pd.concat([df_busquedas, pd.DataFrame([{ 'descripcion': descripcion_buscada }])], ignore_index=True)
df_busquedas.to_csv(busquedas_path, index=False)

# Crear DataFrames finales en memoria
df_validos = pd.DataFrame(validos)
df_descartados = pd.DataFrame(descartados)

# Resumen final
print(f"\n{'='*60}")
print(f"RESUMEN DE SCRAPING")
print(f"{'='*60}")
print(f"Total artículos procesados: {len(lista_articulos)}")
print(f"Productos guardados: {len(productos_guardados)}")
print(f"  - Targets: {sum(1 for p in productos_guardados if p['is_substitute'] == 'target')}")
print(f"  - Sustitutos: {sum(1 for p in productos_guardados if p['is_substitute'] == 'substitute')}")
print(f"Artículos sin match: {len(articulos_sin_match)}")

if articulos_sin_match:
    print(f"\nArtículos que requieren revisión:")
    for art in articulos_sin_match[:10]:
        print(f"  - {art}")

print("\nScraping completado!")
driver.quit()